This notebook is to prototype for a script that performs grid search for filtering narrow rectangles

In [1]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(".."))

All Imports

In [2]:
import numpy as np
import pandas as pd
from config import CONFIG

Directory structue
+ ar_threshold_0.1
++ area_threshold_0
    + scene_00
        + frame_00
            + selected_boxes
            + discarded_boxes
        + frame_01
        + ..
    + scene_01
    + ...
++ area_threshold_10
++ area_threhold_20
++ ...

If you save the filtered bboxes according to this structure, then you can calculate the metrics for a given combination for all frame in all scenes. 

In [3]:
home = os.path.join(os.path.expanduser("~"), CONFIG['HOME_PATH'][CONFIG['OS']])

ps_base_path = os.path.join(home, *CONFIG['BBOX_FILE_PATHS']['ROI'])
if CONFIG['ROI']:
    grid_rect_save_path = os.path.join(home, *CONFIG['GRID_SEARCH_RECT']['PATH']['ROI'])
else:
    grid_rect_save_path = os.path.join(home, *CONFIG['GRID_SEARCH_RECT']['PATH']['FULL_RANGE'])

scene_idx = 0
frame_idx = 0


In [4]:
aspect_ratio_config = CONFIG['GRID_SEARCH_RECT']['ASPECT_RATIO']
area_config = CONFIG['GRID_SEARCH_RECT']['AREA']
rect_ar_range = np.arange(*aspect_ratio_config['RANGE'], aspect_ratio_config['STEP'])
rect_area_range = np.arange(*area_config['RANGE'], area_config['STEP'])

In [5]:
scene_id = os.listdir(ps_base_path)[scene_idx]
scene_path= os.path.join(ps_base_path, scene_id)

frame_id = os.listdir(scene_path)[frame_idx]
frame_path = os.path.join(scene_path, frame_id)

In [22]:
def apply_rect_filter(df, aspect_ratio_col, area_col, min_ratio, min_area):
    """
    Filters bounding boxes based on aspect ratio and area thresholds.

    Args:
        df (pd.DataFrame): DataFrame containing bounding box information.
        aspect_ratio_col (str): Column name for aspect ratios in the DataFrame.
        area_col (str): Column name for areas in the DataFrame.
        min_ratio (float): Minimum aspect ratio for filtering bounding boxes.
        min_area (float): Minimum area for filtering bounding boxes.

    Returns:
        tuple: (selected_boxes, discarded_boxes)
            - selected_boxes (pd.DataFrame): Subset of the DataFrame with bounding boxes that satisfy the conditions.
            - discarded_boxes (pd.DataFrame): Subset of the DataFrame with bounding boxes that do not satisfy the conditions.
    """
    # Filter the DataFrame based on the conditions
    selected_boxes = df[(df[aspect_ratio_col] >= min_ratio) & (df[area_col] >= min_area)]
    discarded_boxes = df[(df[aspect_ratio_col] < min_ratio) | (df[area_col] < min_area)]

    selected_boxes.reset_index(drop=True, inplace=True)
    discarded_boxes.reset_index(drop=True, inplace=True)
    
    return selected_boxes, discarded_boxes


Define functions to filter by aspect ratio and area for a given values of thresholds

In [ ]:
def grid_search_rect_filter(input_path, ar_range, area_range,base_save_path ,config):
    # print(f"Processing {input_frame_path}")
    aspect_ratio_dir = config['GRID_SEARCH_RECT']['AR_THRESHOLD_BASE_DIR_NAME']
    area_dir = config['GRID_SEARCH_RECT']['AREA_THRESHOLD_BASE_DIR_NAME']
    for _, aspect_ratio in enumerate(ar_range):
        ar_dir_name = f"{aspect_ratio_dir}_{aspect_ratio:.1f}"
        
        for _, area in enumerate(area_range):
            area_dir_name = f"{area_dir}_{area}"
            
            for scene in os.listdir(input_path):
                scene_path = os.path.join(input_path, scene)    
                
                for frame in os.listdir(scene_path):
                    frame_id = frame.split(".")[0]
                    frame_path = os.path.join(scene_path, frame)
                    
                    # print(f"Processing {frame_path}")
                    # print(f"Aspect Ratio: {aspect_ratio}, Area: {area}")
                    df = pd.read_feather(frame_path)
                    
                    df['aspect_ratio'] = df['box_width'] /df['box_length']
                    df['area'] = df['box_width'] * df['box_length']
                    selected_boxes_df, discarded_boxes_df = apply_rect_filter(df, 'aspect_ratio', 'area', aspect_ratio, area)
                    
                    # print(selected_boxes_df.head())
                    # print(discarded_boxes_df.head())
                    save_path = os.path.join(base_save_path, ar_dir_name, area_dir_name, scene, frame_id)
                    # print(save_path)
                    os.makedirs(save_path, exist_ok=True)
                    
                    selected_boxes_df.to_feather(os.path.join(save_path, "selected_boxes.feather"))
                    discarded_boxes_df.to_feather(os.path.join(save_path, "discarded_boxes.feather"))

In [37]:
grid_search_rect_filter(ps_base_path, rect_ar_range, rect_area_range, grid_rect_save_path, CONFIG)

'315974909559984000.feather'